In [1]:
import pandas as pd

In [ ]:
movies_df = pd.read_csv('data/ml-25m/movies.csv')
ratings_df = pd.read_csv('data/ml-25m/ratings.csv')

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_info = movies_df.merge(ratings_df, on='movieId')

In [5]:
movies_info.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2,3.5,1141415820
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1439472215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4,3.0,1573944252
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,858625949
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,890492517


In [7]:
user_rating_threshold = 500
active_users = movies_info.groupby('userId')['rating'].count() > user_rating_threshold

In [8]:
active_users_indices = active_users[active_users==True].index

In [9]:
active_users

userId
1         False
2         False
3          True
4         False
5         False
          ...  
162537    False
162538    False
162539    False
162540    False
162541    False
Name: rating, Length: 162541, dtype: bool

In [10]:
movies_watched_by_active_users = movies_info[movies_info['userId'].isin(active_users_indices)]
movie_rating_threshold = 500
popular_movies = movies_watched_by_active_users.groupby('movieId')['rating'].count() > movie_rating_threshold
popular_movie_indices = popular_movies[popular_movies == True].index

final_list_movies = movies_watched_by_active_users[movies_watched_by_active_users['movieId'].isin(popular_movie_indices)]
final_list_movies = final_list_movies.drop_duplicates()

In [11]:
final_list_movies.head()

,movieId,title,genres,userId,rating,timestamp
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1439472215
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12,4.0,1167582601
35,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,120,5.0,956264593
47,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,166,4.0,942014736
49,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,171,4.5,1074594380


In [12]:
final_list_movies['movieId'].nunique()

3950

In [13]:
final_movies_info = final_list_movies[['movieId','title']].drop_duplicates()
user_item = final_list_movies.pivot_table(index='title', columns='userId', values='rating', fill_value=0)

In [14]:
user_item.shape

(3950, 9663)

In [15]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [16]:
def extarct_genres(genres):
    return list(genres.split('|'))

movies_df['genre_list'] = movies_df['genres'].apply(extarct_genres)

In [17]:
genres_list = set(genre for genre_list in movies_df.genre_list for genre in genre_list)

genres_list

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [19]:
list_genres = list(genres_list)[:-1]

In [20]:
import numpy as np

movie_feature_matrix = np.zeros((movies_df.shape[0],len(list_genres)))
for i, genre_list in enumerate(movies_df['genre_list']):
    for j, genre in enumerate(list_genres):
        movie_feature_matrix[i,j] = int(genre in genre_list)

In [21]:
movie_feature_matrix.shape

(62423, 19)